In [1]:
import sys
sys.path.append('../zoho_data_app')

import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Account ID',
'ContactID',
'FirstName',	
'LastName',	
'AddressLine1',	
'AddressLine2',	
'Town',	
'City',	
'Country',
'Account Status']

In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\Member Data\vp\Compile\Sheet NEW\compiled_file.xlsx'
zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1', usecols=cols)

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin

zoho_df.fillna('empty', inplace=True)
zoho_df

In [3]:
zoho_df.loc[zoho_df['ContactID'] == '345709']

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Account Status
123555,100,3,Saurin,Shah,52 PREMIER HOUSE 1 OPP GURUDWARA,S G HIGHWAY,AHMEDABAD,GUJARAT,India,Active


In [4]:
zoho_df.rename(columns={"Town": "Town/City", "City": "State"}, inplace=True)

In [5]:
zoho_df

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town/City,State,Country,Account Status
0,1,1,empty,Pt Ivo Indonesia,empty,empty,empty,empty,empty,Active
1,1,660145,Extra,Detail,empty,empty,empty,empty,empty,Active
2,10,2,Baharudin,Murad,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked
3,10,131077,Siti Suriah,Abdul Rahman,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked
4,10,660146,Extra,Detail,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked
...,...,...,...,...,...,...,...,...,...,...
987864,RTHC,empty,Royal Thailand Holiday Club,empty,empty,empty,empty,empty,empty,Suspended
987865,SFEX,empty,San Francisco Exchange,empty,empty,empty,empty,empty,empty,Suspended
987866,VMI,empty,VMI,empty,empty,empty,empty,empty,empty,Suspended
987867,VOYA,empty,Voyager Club,empty,empty,empty,empty,empty,empty,Suspended


In [6]:
zoho_df['AddressLine1 - NEW'] = zoho_df['AddressLine1']
zoho_df['AddressLine2 - NEW'] = zoho_df['AddressLine2']
zoho_df['Town/City - NEW'] = zoho_df['Town/City']
zoho_df['State - NEW'] = zoho_df['State']
zoho_df['Country - NEW'] = zoho_df['Country']

In [7]:
zoho_df.groupby('ContactID').count()

,Account ID,FirstName,LastName,AddressLine1,AddressLine2,Town/City,State,Country,Account Status,AddressLine1 - NEW,AddressLine2 - NEW,Town/City - NEW,State - NEW,Country - NEW
ContactID,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,2,2,2,2,2,2,2,2,2,2,2,2,2,2
100,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999996,1,1,1,1,1,1,1,1,1,1,1,1,1,1
999997,1,1,1,1,1,1,1,1,1,1,1,1,1,1
999999,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# ====================== Awal Filter Umum ====================== 

In [8]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['ContactID'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['ContactID'])]

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town/City,State,Country,Account Status,AddressLine1 - NEW,AddressLine2 - NEW,Town/City - NEW,State - NEW,Country - NEW
13,1000056,921274,Yohanes,Mustamu,WELLINGTON RESIDENCE BLOK A1,"RT. 009/009 BENDA BARU, PAMULANG",TANGERANG SELATAN,BANTEN,Indonesia,Active,WELLINGTON RESIDENCE BLOK A1,"RT. 009/009 BENDA BARU, PAMULANG",TANGERANG SELATAN,BANTEN,Indonesia
63,1000434,921850,John David,Walters,"\""BUNCHERS\"" NORTH COMMON",empty,REDBOURN,HERTFORDSHIRE,United Kingdom,Active,"\""BUNCHERS\"" NORTH COMMON",empty,REDBOURN,HERTFORDSHIRE,United Kingdom
72,1000438,921859,Edwin,Wijaya,condominium green bay tower m lantai 31 no aC,-,jakarta utara,dki jakarta,Indonesia,Active,condominium green bay tower m lantai 31 no aC,-,jakarta utara,dki jakarta,Indonesia
75,1000489,921955,-,Rosameris,JL. SEI MENCIRIM NO. 02 MEDAN,"BABURA, MEDAN BARU",MEDAN,SUMATERA UTARA,Indonesia,Active,JL. SEI MENCIRIM NO. 02 MEDAN,"BABURA, MEDAN BARU",MEDAN,SUMATERA UTARA,Indonesia
83,10005,10,Leandro Ladislao G.,Roquel,27 CADENA DE AMOR STREET,"DE CASTRO SUBDIVISION, STA. LUCIA",PASIG CITY,MM,Philippines,Revoked,27 CADENA DE AMOR STREET,"DE CASTRO SUBDIVISION, STA. LUCIA",PASIG CITY,MM,Philippines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987864,RTHC,empty,Royal Thailand Holiday Club,empty,empty,empty,empty,empty,empty,Suspended,empty,empty,empty,empty,empty
987865,SFEX,empty,San Francisco Exchange,empty,empty,empty,empty,empty,empty,Suspended,empty,empty,empty,empty,empty
987866,VMI,empty,VMI,empty,empty,empty,empty,empty,empty,Suspended,empty,empty,empty,empty,empty
987867,VOYA,empty,Voyager Club,empty,empty,empty,empty,empty,empty,Suspended,empty,empty,empty,empty,empty


In [9]:
zoho_df.drop_duplicates(subset=['ContactID'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town/City,State,Country,Account Status,AddressLine1 - NEW,AddressLine2 - NEW,Town/City - NEW,State - NEW,Country - NEW
0,1,1,empty,Pt Ivo Indonesia,empty,empty,empty,empty,empty,Active,empty,empty,empty,empty,empty
1,1,660145,Extra,Detail,empty,empty,empty,empty,empty,Active,empty,empty,empty,empty,empty
2,10,2,Baharudin,Murad,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore
3,10,131077,Siti Suriah,Abdul Rahman,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore
4,10,660146,Extra,Detail,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore,Revoked,BLK 829,"JURONG WEST ST. 81, # 08-308",empty,empty,Singapore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942089,999996,921189,Bawanta,Widya Suta,-,empty,-,-,Indonesia,Active,-,empty,-,-,Indonesia
942090,999997,921190,Rahmat,Ichram Silalahi,"Taman Kenari Blok XG/12 RT. 004/012 Puspasari,...",empty,Bogor,Jawa Barat,Indonesia,Active,"Taman Kenari Blok XG/12 RT. 004/012 Puspasari,...",empty,Bogor,Jawa Barat,Indonesia
942091,999997,921198,Eira,Aurelia Hollanda,"Taman Kenari Blok XG/12 RT. 004/012 Puspasari,...",empty,Bogor,Jawa Barat,Indonesia,Active,"Taman Kenari Blok XG/12 RT. 004/012 Puspasari,...",empty,Bogor,Jawa Barat,Indonesia
942092,999998,921191,Henny,Herawati,-,empty,-,-,Indonesia,Active,-,empty,-,-,Indonesia


In [10]:
df_india = zoho_df.loc[zoho_df['Country'].str.contains('india', regex=True, flags=re.I)].copy()
df_australia = zoho_df.loc[zoho_df['Country'].str.contains('australia', regex=True, flags=re.I)].copy()
df_indonesia = zoho_df.loc[zoho_df['Country'].str.contains('indonesia', regex=True, flags=re.I)].copy()
df_row = zoho_df.loc[~zoho_df['Country'].str.contains('india|indonesia|australia', regex=True, flags=re.I)].copy()

count_df_india = df_india.shape[0]
count_df_australia = df_australia.shape[0]
count_df_indonesia = df_indonesia.shape[0]
count_df_row = df_row.shape[0]

df_india.reset_index(drop=True, inplace=True)
df_australia.reset_index(drop=True, inplace=True)
df_indonesia.reset_index(drop=True, inplace=True)
df_row.reset_index(drop=True, inplace=True)

df_india.replace('empty', '', inplace=True)
df_australia.replace('empty', '', inplace=True)
df_indonesia.replace('empty', '', inplace=True)
df_row.replace('empty', '', inplace=True)

df_india['No'] = df_india.index
df_australia['No'] = df_australia.index
df_indonesia['No'] = df_indonesia.index
df_row['No'] = df_row.index

count_total_all = count_df_india+count_df_australia+count_df_indonesia+count_df_row

In [11]:
df_india.loc[df_india['ContactID'] == '3']

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town/City,State,Country,Account Status,AddressLine1 - NEW,AddressLine2 - NEW,Town/City - NEW,State - NEW,Country - NEW,No
871,100,3,Saurin,Shah,52 PREMIER HOUSE 1 OPP GURUDWARA,S G HIGHWAY,AHMEDABAD,GUJARAT,India,Active,52 PREMIER HOUSE 1 OPP GURUDWARA,S G HIGHWAY,AHMEDABAD,GUJARAT,India,871


In [124]:
new_columns = ['No', 'Account ID', 'ContactID', 'FirstName', 'LastName', 'AddressLine1', 'AddressLine1 - NEW',
       'AddressLine2', 'AddressLine2 - NEW', 'Town/City', 'Town/City - NEW', 'State', 'State - NEW', 'Country', 'Country - NEW', 'Account Status']

df_india = df_india.reindex(columns=new_columns)
df_australia = df_australia.reindex(columns=new_columns)
df_indonesia = df_indonesia.reindex(columns=new_columns)
df_row = df_row.reindex(columns=new_columns)

In [125]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data India : "+str(count_df_india))
print("Jumlah Data Australia : "+str(count_df_australia))
print("Jumlah Data Indonesia : "+str(count_df_indonesia))
print("Jumlah Data ROW : "+str(count_df_row))
print("Jumlah Data Total : "+str(count_total_all))

Summary Report
Jumlah Data India : 514219
Jumlah Data Australia : 89358
Jumlah Data Indonesia : 95814
Jumlah Data ROW : 242703
Jumlah Data Total : 942094


In [126]:
df_india.to_excel(r'C:\Users\fajar\Documents\Python\Data\Member Data\vp\Compile\Sheet NEW\Country Based\df_india.xlsx', index=False)
df_australia.to_excel(r'C:\Users\fajar\Documents\Python\Data\Member Data\vp\Compile\Sheet NEW\Country Based\df_australia.xlsx', index=False)
df_indonesia.to_excel(r'C:\Users\fajar\Documents\Python\Data\Member Data\vp\Compile\Sheet NEW\Country Based\df_indonesia.xlsx', index=False)
df_row.to_excel(r'C:\Users\fajar\Documents\Python\Data\Member Data\vp\Compile\Sheet NEW\Country Based\df_row.xlsx', index=False)

In [127]:
df_india.loc[df_india['ContactID'] == '21112']

,No,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine1 - NEW,AddressLine2,AddressLine2 - NEW,Town/City,Town/City - NEW,State,State - NEW,Country,Country - NEW,Account Status
107353,107353,1751,21112,Ashish,Desal,11 ARCHAN APARTMENTS,11 ARCHAN APARTMENTS,24 PANCHVANTI SOCIETY MANINAGAR,24 PANCHVANTI SOCIETY MANINAGAR,AHMEDABAD,AHMEDABAD,GUJ,GUJ,India,India,Revoked
